# Pandas
-----
 - Where to get help from? Stack Overflow!
 - Reference books: Python for Data Analyst by O'reilly, Learning the Pandas Library by Matt Harrison
 - Planet Python: https://planetpython.org/
 - Data Skeptic podcast: https://dataskeptic.com/





-------
# The Series Data Structure
-------
 - The series is one of the core data structures in pandas. You think of it across between a list and a dictionary. 

In [ ]:
import pandas as pd

pd.Series? # you can see the documentation, you can pass in some data, an index, and a name.


In [ ]:
animals = ['Tiger', 'Bear', 'Moose']
pd.Series(animals)

In [ ]:
numbers = [1, 2, 3]
pd.Series(numbers)

 - Underneath panda stores series values in a typed array using the Numpy library. This offers significant speed-up when processing data versus traditional python lists. 
 - Underneath, pandas does some type conversion. If we create a list of strings and we have one element, a None type, pandas inserts it as a None and uses the type object for the underlying array. 

In [ ]:
animals = ['Tiger', 'Bear', None]
pd.Series(animals)

 - NAN is not none and when we try the equality test, it's false. 
 - You need to use special functions to test for the presence of not a number, such as the Numpy library `isnan`. 

In [ ]:
numbers = [1, 2, None]
pd.Series(numbers)

In [ ]:
import numpy as np
np.nan == None

In [ ]:
np.nan == np.nan

In [ ]:
np.isnan(np.nan)

 - A series can be created from dictionary data. If you do this, the index is automatically assigned to the keys of the dictionary that you provided and not just incrementing integers.   

In [ ]:
sports = {'Archery': 'Bhutan',
          'Golf': 'Scotland',
          'Sumo': 'Japan',
          'Taekwondo': 'South Korea'}
s = pd.Series(sports)
s

 - Once the series has been created, we can get the index object using the index attribute. 

In [ ]:
s.index

 - You could also separate your index creation from the data by passing in the index as a list explicitly to the series. 

In [ ]:
s = pd.Series(['Tiger', 'Bear', 'Moose'], index=['India', 'America', 'Canada'])
s

 - Pandas overrides the automatic creation to favor only and all of the indices values that you provided. So it will ignore it from your dictionary, all keys, which are not in your index, and pandas will add non type or NAN values for any index value you provide, which is not in your dictionary key list. 

In [ ]:
sports = {'Archery': 'Bhutan',
          'Golf': 'Scotland',
          'Sumo': 'Japan',
          'Taekwondo': 'South Korea'}
s = pd.Series(sports, index=['Golf', 'Sumo', 'Hockey'])
s

-------
# Quering a Series
-------

In [ ]:
sports = {'Archery': 'Bhutan',
          'Golf': 'Scotland',
          'Sumo': 'Japan',
          'Taekwondo': 'South Korea'}
s = pd.Series(sports)
s

 - To query by numeric location, starting at zero, use the `iloc` attribute. 

In [ ]:
s.iloc[3]

 - To query by the index label, you can use the `loc` attribute.
 - Keep in mind that `iloc` and `loc` are not methods, they are attributes.

In [ ]:
s.loc['Golf']

In [ ]:
s['Golf']

 - So what happens if your index is a list of integers? This is a bit complicated, and Pandas can't determine automatically whether you're intending to query by index position or index label. 

In [ ]:
sports = {99: 'Bhutan',
         100: 'Scotland',
         101: 'Japan',
         102: 'South Korea'}
s = pd.Series(sports)

In [ ]:
s[0] # This won't call s.iloc[0] as one might expect, it generates an error instead

In [ ]:
s.iloc[0]

 - Let's talk about working with the data. A common task is to want to consider all of the values inside of a series and want to do some sort of operation. 

In [ ]:
s = pd.Series([100.00, 120.00, 101.00, 3.00])
s

 - We could write a little routine which iterates over all of the items in the series and adds them together to get a total. This works, but it's slow.

In [ ]:
total = 0
for item in s:
    total += item
print(total)

 - Pandas and the underlying NumPy libraries support a method of computation called vectorization. 
 - we just call `np.sum` and pass in an iterable item. 

In [ ]:
import numpy as np

total = np.sum(s)
print(total)

 - `head` method reduces the amount of data printed out by the series to the first five elements. 

In [ ]:
# This creats a big series of random numbers
s = pd.Series(np.random.randint(0,1000,10000))
s.head()

In [ ]:
len(s)

#### `timeit`
- let's see if the second solution is actually faster than the other one? we can examin this with a magic function! :D
- Magic functions begin with a % sign. If we type this sign and then hit the Tab key, we can see a list of the available magic functions. 
- We're actually going to use what's called a cellular magic function. These start with two percentage signs.`%%timeit`
- You can give timeit the number of loops that you would like to run. By default, we'll use 1,000 loops. 

In [ ]:
%%timeit -n 100
summary = 0
for item in s:
    summary += item

In [ ]:
%%timeit -n 100
summary = np.sum(s)

 - Related feature in Pandas and NumPy is called broadcasting. With broadcasting, you can apply an operation to every value in the series, changing the series. 

In [ ]:
s += 2 #adds two to each item in s using broadcasting
s.head()

In [ ]:
for label, value in s.iteritems():
    s.at[label] = value + 2
s.head()

In [ ]:
%%timeit -n 10
s = pd.Series(np.random.randint(0, 1000, 10000))
for label, value in s.iteritems():
    s.loc[label] = value + 2

In [ ]:
%%timeit -n 10
s = pd.Series(np.random.randint(0, 1000, 10000))
s += 2

 - The .loc attribute lets you not only modify data in place, but also add new data as well. If the value you pass in as the index doesn't exist, then a new entry is added. And keep in mind, indices can have mixed types.
 - While it's important to be aware of the typing going on underneath, Pandas will automatically change the underlying NumPy types as appropriate. 

In [ ]:
s = pd.Series([1, 2, 3])
s.loc['Animal'] = 'Bears'
s

 - There are a couple of important considerations when using append:
     1. Pandas is going to take your series and try to infer the best data types to use. 
     2. the append method doesn't actually change the underlying series. It instead returns a new series which is made up of the two appended together. 
     (This is actually a significant issue for new Pandas users who are used to objects being changed in place. So watch out for it, not just with append but with other Pandas functions as well.)
     3. we see that when we query the appended series for those who have cricket as their national sport, we don't get a single value, but a series itself. This is actually very common, and if you have a relational database background, this is very similar to every table query resulting in a return set which itself is a table. 
     

In [ ]:
original_sports = pd.Series({'Archery': 'Bhutan',
                            'Golf': 'Scotland',
                            'Sumo': 'Japan',
                            'Taekwando': 'South Korea'})
cricket_loving_countries = pd.Series(['Autralia',
                                     'Barbados',
                                     'Pakistan',
                                     'England'],
                                    index=['Cricket',
                                          'Cricket',
                                          'Cricket',
                                          'Cricket'])
all_countries = original_sports.append(cricket_loving_countries)

In [ ]:
original_sports

In [ ]:
cricket_loving_countries

In [ ]:
all_countries

-------
# The DataFrame Data Structure
-------
 - The DataFrame data structure is the heart of the Panda's library. It's a primary object that you'll be working with in data analysis and cleaning tasks. 
 - The DataFrame is conceptually a two-dimensional series object, where there's an index and multiple columns of content, with each column having a label. 
 - In fact, the distinction between a column and a row is really only a conceptual distinction. And you can think of the DataFrame itself as simply a two-axes labeled array. 

In [ ]:
import pandas as pd
purchase_1 = pd.Series({'Name': 'Chris',
                       'Item Purchased': 'Dog Food',
                       'Cost': 22.50})
purchase_2 = pd.Series({'Name': 'Kevin',
                       'Item Purchased': 'Kitty Litter',
                       'Cost': 2.50})
purchase_3 = pd.Series({'Name': 'Vinod',
                       'Item Purchased': 'Bird Seed',
                       'Cost': 5.00})
df = pd.DataFrame([purchase_1, purchase_2, purchase_3], index=['Store 1', 'Store 1', 'Store 2'])
df.head()

 - Because the DataFrame is two-dimensional, passing a single value to the `loc` indexing operator will return series if there's only one row to return. 

In [ ]:
df.loc['Store 2']

In [ ]:
type(df.loc['Store 2'])

 - It's important to remember that the indices and column names along either axes, horizontal or vertical, could be non-unique. 
 - For instance, in this example, we see two purchase records for Store 1 as different rows. If we use a single value with the DataFrame `loc` attribute, multiple rows of the DataFrame will return, not as a new series, but as a new DataFrame. 

In [ ]:
df.loc['Store 1']

 - One of the powers of the Panda's DataFrame is that you can quickly select data based on multiple axes. 

In [ ]:
df.loc['Store 1', 'Cost']

 - What if we just wanted to do column selection and just get a list of all of the costs? 
     1. First, you can get a transpose of the DataFrame, using the capital T attribute, which swaps all of the columns and rows. This essentially turns your column names into indices. And we can then use the `loc` method. This works, but it's pretty ugly:

In [ ]:
df.T

In [ ]:
df.T.loc['Cost']

 - Since iloc and loc are used for row selection, the Panda's developers reserved indexing operator directly on the DataFrame for column selection. 
 - In a Panda's DataFrame, columns always have a name. So this selection is always label based
     2. So the second way to do it is by simply using indexing operator:
 

In [ ]:
df['Cost']

 - You can also chain operations together.
 - Chaining can come with some costs and is best avoided if you can use another approach. In particular, chaining tends to cause Pandas to return a copy of the DataFrame instead of a view on the DataFrame. 
 - For selecting a data, this is not a big deal, though it might be slower than necessary. If you are changing data though, this is an important distinction and can be a source of error. 

In [ ]:
df.loc['Store 1']['Cost']

 - `.loc` also supports slicing. If we wanted to select all rows, we can use a column to indicate a full slice from beginning to end. And then add the column name as the second parameter as a string. In fact, if we wanted to include multiply columns, we could do so in a list. And Pandas will bring back only the columns we have asked for. 

In [ ]:
df.loc[:, ['Name', 'Cost']]

 - It's easy to delete data in series and DataFrames, and we can use the `drop` function to do so. 
 - This function takes a single parameter, which is the index or roll label, to drop. 
 - The drop function doesn't change the DataFrame by default. And instead, returns to you a copy of the DataFrame with the given rows removed. 

In [ ]:
df.drop('Store 1')

In [ ]:
df

 - Let's make a copy with the copy method and do a drop on it instead. This is a very typical pattern in Pandas, where in place changes to a DataFrame are only done if need be, usually on changes involving indices. 

In [ ]:
copy_df = df.copy()
copy_df = copy_df.drop('Store 1')
copy_df

 - Drop has two interesting optional parameters. The first is called in place, and if it's set to true, the DataFrame will be updated in place, instead of a copy being returned. 
 - The second parameter is the axes, which should be dropped. By default, this value is 0, indicating the row axes. But you could change it to 1 if you want to drop a column. 

In [ ]:
copy_df.drop?

 - There is a second way to drop a column, however. And that's directly through the use of the indexing operator, using the `del` keyword. 
 - This way of dropping data, however, takes immediate effect on the DataFrame and does not return a view. 

In [ ]:
del copy_df['Name']
copy_df

 - Finally, adding a new column to the DataFrame is as easy as assigning it to some value. For instance, if we wanted to add a new location as a column with default value of none, we could do so by using the assignment operator after the square brackets. This broadcasts the default value to the new column immediately. 

In [ ]:
df['Location'] = None
df

------
# Dataframe Indexing and Loading
------
 - The common work flow is to read your data into a DataFrame then reduce this DataFrame to the particular columns or rows that you're interested in working with. 
 - The Panda's toolkit tries to give you views on a DataFrame. This is much faster than copying data and much more memory efficient too. 
 - But it does mean that if you're manipulating the data you have to be aware that any changes to the DataFrame you're working on may have an impact on the base data frame you used originally. 

In [ ]:
costs = df['Cost']
costs

In [ ]:
costs += 2
costs

In [ ]:
df

- Pandas has built-in support for delimited files such as CSV files as well as a variety of other data formats including relational databases, Excel, and HTML tables. 
- We've saved a CSV file called olympics.csv, which has data from Wikipedia that contains a summary list of the medal various countries have won at the Olympics. 
- We can take a look at this file using the shell command `cat`. Which we can invoke directly using the exclamation point. 
- What happens here is that when the Jupyter notebook sees a line beginning with an exclamation mark, it sends the rest of the line to the operating system shell for evaluation. 

In [ ]:
!cat olympics.csv

- We can read this into a DataFrame by calling the `read_csv` function of the module. 
- When we look at the DataFrame we see that the first cell has an NaN in it since it's an empty value, and the rows have been automatically indexed for us. 
- It seems pretty clear that the first row of data in the DataFrame is what we really want to see as the column names. It also seems like the first column in the data is the country name, which we would like to make an index. 

In [ ]:
df = pd.read_csv('olympics.csv')
df.head()

- `read_csv` has a number of parameters that we can use to indicate to Pandas how rows and columns should be labeled. 
- For instance, we can use the `index_col` to indicate which column should be the index and we can also use the header parameter to indicate which row from the data file should be used as the header. 
- Let's re-import that data and center index value to be 0 which is the first column and let set a column headers to be read from the second row of data. We can do this by using the `skiprows` parameters, to tell Pandas to ignore the first row, which was made up of numeric column names. 

In [ ]:
df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)
df.head()

- this labeling isn't really as clear as it could be, so we should clean up the data file. 
- Panda stores a list of all of the columns in the `.columns` attribute. We can change the values of the column names by iterating over this list and calling the `rename` method of the data frame. 

In [ ]:
df.columns

- Here we just iterate through all of the columns looking to see if they start with a 01, 02, 03 or numeric character. If they do, we can call `rename` and set the column parameters to a dictionary with the keys being the column we want to replace and the value being the new value we want. 
- Here we'll slice some of the old values in two, since we don't want to lose the unique appended values. We'll also set the ever-important `inplace` parameter to true so Pandas knows to update this data frame directly. 

In [ ]:
for col in df.columns:
    if col[:2] == '01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2] == '02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2] == '03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1] == '№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

df.head()

-----
# Quering a DataFrame
-----
- We are going to talk about boolean masking.
- Boolean masking is the heart of fast and efficient querying in NumPy. It's analogous a bit to masking used in other computational areas. 
- A Boolean mask is an array which can be of one dimension like a series, or two dimensions like a data frame, where each of the values in the array are either true or false. 
- This array is essentially overlaid on top of the data structure that we're querying. 
- And any cell aligned with the true value will be admitted into our final result, and any sign aligned with a false value will not. 
- Boolean masks are created by applying operators directly to the pandas series or DataFrame objects. 

- To build a Boolean mask for the query of which country achieved a gold medal at the summer Olympic, we project the gold column using the indexing operator and apply the greater than operator with a comparison value of zero. 
- This is essentially broadcasting a comparison operator, greater than, with the results being returned as a Boolean series. 

In [ ]:
df['Gold'] > 0

- So this builds us the Boolean mask, which is half the battle. What we want to do next is overlay that mask on the data frame. 
- We can do this using the `where` function. The `where` function takes a Boolean mask as a condition, applies it to the data frame or series, and returns a new data frame or series of the same shape. 
- The result would be a data frame of only those countries who have won a gold at a summer games. 
- All of the countries which did not meet the condition have NaN data instead. Most statistical functions built into the data frame object ignore values of NaN. 

In [ ]:
only_gold = df.where(df['Gold'] > 0)
only_gold.head()

In [ ]:
only_gold['Gold'].count()

In [ ]:
df['Gold'].count()

- Often we want to drop those rows which have no data. To do this, we can use the `dropna` function. 
- You can optionally provide `dropna` the axes it should be considering.
- Remember that the axes is just an indicator for the columns or rows and that the default is zero, which means rows. 

In [ ]:
only_gold = only_gold.dropna()
only_gold.head()

- In pandas we don't actually have to use the where function explicitly. The pandas developers allow the indexing operator to take a Boolean mask as a value instead of just a list of column names. 
- You'll notice that there are no NaNs when you query the data frame in this manner. pandas automatically filters out the rows with now values. 

In [ ]:
only_gold = df[df['Gold'] > 0]
only_gold.head()

- The output of two Boolean masks being compared with logical operators is another Boolean mask. This means that you can chain together a bunch of and/or statements in order to create more complex queries, and the result is a single Boolean mask. 
- For instance, we could create a mask for all of those countries who have received a gold in the summer Olympics and logically order that with all of those countries who have received a gold in the winter Olympics. 

In [ ]:
len(df[(df['Gold']>0) | (df['Gold.1']>0)])

- As another example, Have there been any countries who have only won a gold in the winter Olympics and never in the summer Olympics?

In [ ]:
df[(df['Gold.1']>0) & (df['Gold']==0)]

- This is important to remember, that each Boolean mask needs to be encased in parenthesis because of the order of operations. This can cause no end of frustration if you're not used to it, so be careful. 

------
# Indexing DataFrames
------
- both series and DataFrames can have indices applied to them. The index is essentially a row level label, and we know that rows correspond to axis zero. 
- Indices can either be inferred or we can set them by the `set_index` function. This function takes a list of columns and promotes those columns to an index.
- `set_index` is a destructive process, it doesn't keep the current index. If you want to keep the current index, you need to manually create a new column and copy into it values from the index attribute. 

In [ ]:
df.head()

- Let's go back to our Olympics DataFrame. Let's say that we don't want to index the DataFrame by countries, but instead want to index by the number of gold medals that were won at summer games. 
- First we need to preserve the country information into a new column. We can do this using the indexing operator or the string that has the column label. Then we can use the `set_index` to set index of the column to summer gold medal wins. 

In [ ]:
df['country'] = df.index
df = df.set_index('Gold')
df.head()

- We can get rid of the index completely by calling the function `reset_index`. This promotes the index into a column and creates a default numbered index. 

In [ ]:
df = df.reset_index()
df.head()

- One nice feature of pandas is that it has the option to do multi-level indexing. This is similar to composite keys in relational database systems. To create a multi-level index, we simply call `set_index` and give it a list of columns that we're interested in promoting to an index. Pandas will search through these in order, finding the distinct data and forming composite indices. 

- Let's change data sets and look at some census data for a better example.
- in this data set there are two summarized levels, one that contains summary data for the whole country. And one that contains summary data for each state, and one that contains summary data for each county. 

In [ ]:
df = pd.read_csv('census.csv')
df.head()

- We often find that we want to see a list of all the unique values in a given column. In this DataFrame, we see that the possible values for the sum level are using the `unique` function on the DataFrame. This is similar to the SQL distinct operator. 
- Here we can run `unique` on the sum level of our current DataFrame and see that there are only two different values, 40 and 50. 

In [ ]:
df['SUMLEV'].unique()

In [ ]:
df = df[df['SUMLEV']==50]
df.head()

- Let's get rid of all of the rows that are summaries at the state level and just keep the county data. Also while this data set is interesting for a number of different reasons, let's reduce the data that we're going to look at to just the total population estimates and the total number of births. 
- We can do this by creating a list of column names that we want to keep then project those and assign the resulting DataFrame to our df variable. 

In [ ]:
columns_to_keep = ['STNAME',
                  'CTYNAME',
                  'BIRTHS2010',
                  'BIRTHS2011',
                  'BIRTHS2012',
                  'BIRTHS2013',
                  'BIRTHS2014',
                  'BIRTHS2015',
                  'POPESTIMATE2010',
                  'POPESTIMATE2011',
                  'POPESTIMATE2012',
                  'POPESTIMATE2013',
                  'POPESTIMATE2014',
                  'POPESTIMATE2015']
df = df[columns_to_keep]
df.head()

- We can load the data and set the index to be a combination of the state and county values and see how pandas handles it in a DataFrame.
- We see here that we have a dual index, first the state name and then the county name. 

In [ ]:
df = df.set_index(['STNAME', 'CTYNAME'])
df.head()

- How we can query this DataFrame?
- When you use a MultiIndex, you must provide the arguments in order by the level you wish to query. 
- Inside of the index, each column is called a level and the outermost column is level zero. 
- For instance, if we want to see the population results from Washtenaw County, you'd want to the first argument as the state of Michigan. 

In [ ]:
df.loc['Michigan', 'Washtenaw County']

- You might be interested in just comparing two counties. For instance, Washtenaw where I live and Wayne County which covers Detroit. To do this, we can pass the `loc` method, a list of tuples which describe the indices we wish to query. Since we have a MultiIndex of two values, the state and the county, we need to provide two values as each element of our filtering list. 

In [ ]:
df.loc[[('Michigan', 'Washtenaw County'),
        ('Michigan', 'Wayne County')]]

------
# Missing values
------
- We've seen a preview of how Pandas handles missing values using the None type and NumPy NaN values. Missing values are pretty common in data cleaning activities.

- The log.csv are logs from online learning systems. In these systems it's common for the player for have a heartbeat functionality where playback statistics are sent to the server every so often, maybe every 30 seconds. 
- These heartbeats can get big as they can carry the whole state of the playback system, such as where the video play head is at, where the video size is, which video is being rendered to the screen, how loud the volume is, etc. 

- In this data the first column is a timestamp in the Unix epoch format. The next column is the user name followed by a web page they're visiting and the video that they're playing. 
- Each row of the DataFrame has a playback position. And we can see that as the playback position increases by one, the time stamp increases by about 30 seconds. 
- Except for user Bob. It turns out that Bob has paused his playback so as time increases the playback position doesn't change. 
- There are a lot of missing values in the paused and volume columns. It's not efficient to send this information across the network if it hasn't changed. So this particular system just inserts null values into the database if there's no changes. 

In [ ]:
df = pd.read_csv('log.csv')
df

- One of the handy functions that Pandas has for working with missing values is the filling function, `fillna`. 
- This function takes a number or parameters, for instance, you could pass in a single value which is called a scalar value to change all of the missing data to one value.
- Next one is the method parameter. The two common fill values are `ffill` and `bfill`. `ffill` is for forward filling and it updates an NaN value for a particular cell with the value from the previous row. 
- It's important to note that your data needs to be sorted in order for this to have the effect you might want. Data that comes from traditional database management systems usually has no order guarantee

In [ ]:
df.fillna?

- In Pandas we can sort either by index or by values. Here we'll just promote the time stamp to an index then sort on the index. 
- If we look closely at the output though we'll notice that the index isn't really unique. Two users seem to be able to use the system at the same time. Again, a very common case. 

In [ ]:
df = df.set_index('time')
df = df.sort_index()
df

- Let's reset the index, and use some multi-level indexing instead, and promote the user name to a second level of the index to deal with that issue. 

In [ ]:
df = df.reset_index()
df = df.set_index(['time', 'user'])
df

- Now that we have the data indexed and sorted appropriately, we can fill the missing datas using `ffill`.

In [ ]:
df = df.fillna(method='ffill')
df.head()

- It's sometimes useful to use forward filling, sometimes backwards filling, and sometimes useful to just use a single number. 
- More recently, the Pandas team introduced a method of filling missing values with a series which is the same length as your DataFrame. This makes it easy to derive values which are missing if you have the underlying to do so. 
- For instance, if you're dealing with receipts and you have a column for final price and a column for discount but are missing information from the original price column, you can fill this automatically using `fillna`. 
- One last note on missing values. When you use statistical functions on DataFrames, these functions typically ignore missing values. For instance if you try and calculate the mean value of a DataFrame, the underlying NumPy function will ignore missing values. This is usually what you want but you should be aware that values are being excluded. 